# Debiased Machine Learning - The Testing Converge Hypothesis Lab (Barro-Lee Growth Data)

![](https://www.nasdaq.com/sites/acquia.prod/files/styles/720x400/public/2019/11/06/investing11.jpg?h=a338d484&itok=mFGvI1Hj)

We will use the DML algorithm we have learned using The Testing Convergence Hypothesis Lab (Barro-Lee growth data) and the next variables for the main analysis:

$y$ = outcome: growth rate

$d$ = treatment: initial wealth

$x$ = controls: country characteristics

We will run the next regressions:

- OLS without including the country characteristics.
- OLS including the country characteristics.
- DML using Lasso to predict y an d.
- DML using Post-Lasso to predict y an d.
- DML using Elastic Net to predict y an d.
- DML using Ridge to predict y an d.
- DML using Random Forest to predict y an d.  

We will run the best method i.e. the best combination of methods to predict y an d., and show our results in a table.

In [70]:
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense


In [71]:
# Import relevant packages
import pandas as pd
import numpy as np
import pyreadr
from sklearn import preprocessing
import patsy

from numpy import loadtxt

import math
import hdmpy
import numpy as np
import random
import statsmodels.api as sm
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import colors
import warnings
warnings.filterwarnings('ignore')

In [72]:
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeCV, ElasticNetCV
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
import itertools
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
from pandas.api.types import is_categorical_dtype
from itertools import compress
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.feature_selection import SelectFromModel

<div class="alert alert-block alert-success">
<b>Success:</b> First me import the packages.
</div>

-  Load the data

In [77]:
# I downloaded the data that the author used
growth_read = pyreadr.read_r('D:/2021-2/estadística_aplicada/data/GrowthData.RData')

# Extracting the data frame from rdata_read
data = growth_read[ 'GrowthData' ]



In [78]:
data.head(5)

,Outcome,intercept,gdpsh465,bmp1l,freeop,freetar,h65,hm65,hf65,p65,...,seccf65,syr65,syrm65,syrf65,teapri65,teasec65,ex1,im1,xr65,tot1
0,-0.024336,1,6.591674,0.2837,0.153491,0.043888,0.007,0.013,0.001,0.29,...,0.04,0.033,0.057,0.010,47.6,17.3,0.0729,0.0667,0.348,-0.014727
1,0.100473,1,6.829794,0.6141,0.313509,0.061827,0.019,0.032,0.007,0.91,...,0.64,0.173,0.274,0.067,57.1,18.0,0.0940,0.1438,0.525,0.005750
2,0.067051,1,8.895082,0.0000,0.204244,0.009186,0.260,0.325,0.201,1.00,...,18.14,2.573,2.478,2.667,26.5,20.7,0.1741,0.1750,1.082,-0.010040
3,0.064089,1,7.565275,0.1997,0.248714,0.036270,0.061,0.070,0.051,1.00,...,2.63,0.438,0.453,0.424,27.8,22.7,0.1265,0.1496,6.625,-0.002195
4,0.027930,1,7.162397,0.1740,0.299252,0.037367,0.017,0.027,0.007,0.82,...,2.11,0.257,0.287,0.229,34.5,17.6,0.1211,0.1308,2.500,0.003283


<div class="alert alert-block alert-success">
<b>Success:</b> Data loaded
</div>

<div class="alert alert-block alert-info">
<b>Tip:</b> Now we have to set the main variables</div> 

- y = outcome: growth rate

- d = treatment: initial wealth

- x = controls: country characteristics

In [79]:
y = data['Outcome']
d = data['gdpsh465']
x = data.drop(['Outcome','gdpsh465','intercept'], 1)

## 1. OLS without including the country characteristics.


In [80]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy

- We made the model equation without control variables

In [81]:
model = "y ~ d"
baseline_ols = smf.ols(model1 , data=data).fit()
baseline_ols_table = baseline_ols.summary2().tables[1]
print( baseline_ols_table.iloc[ 1 , 4:] )
baseline_ols_table.iloc[1, :]

[0.025   -0.010810
0.975]    0.013444
Name: d, dtype: float64


Coef.       0.001317
Std.Err.    0.006102
t           0.215777
P>|t|       0.829661
[0.025     -0.010810
0.975]      0.013444
Name: d, dtype: float64

In [106]:
baseline_ols.summary2().tables[1]['Coef.']['d']

0.001316712613446065

In [82]:
baseline_ols_table.loc[['d'],:]

,Coef.,Std.Err.,t,P>|t|,[0.025,0.975]
d,0.001317,0.006102,0.215777,0.829661,-0.01081,0.013444


<div class="alert alert-block alert-success">
<b>Success:</b> Model runned
</div>

An increase by 1% in the initial wealth cause an increase in the outcome by 0.001317. This model is without the country characteristics and says that these variables are low related.

***

## 2. OLS including the country characteristics.

- We select only the country characteristics variables to include in the model equation
- Also run the model including the country characteristics

In [83]:
control_formula = "y~ d + x"

In [84]:
control_ols = smf.ols( control_formula , data=data).fit()
control_ols_table = control_ols.summary2().tables[1]
print( control_ols_table.iloc[ 1 , 4:] )
control_ols_table.iloc[1, :]

[0.025   -0.070600
0.975]    0.051844
Name: d, dtype: float64


Coef.      -0.009378
Std.Err.    0.029888
t          -0.313774
P>|t|       0.756019
[0.025     -0.070600
0.975]      0.051844
Name: d, dtype: float64

In [85]:
control_ols_table.loc[['d'],:]

,Coef.,Std.Err.,t,P>|t|,[0.025,0.975]
d,-0.009378,0.029888,-0.313774,0.756019,-0.0706,0.051844


This can interpret as if the initial wealth increase by 1%, then the outcome decrease by -0.0093. This model has all the country characteristics in the equation. 

<div class="alert alert-block alert-success">
<b>Success:</b> Model runned
</div>

## DML algorithm

Here we perform inference of the predictive coefficient $\beta$ in our partially linear statistical model, 

$$
Y = D\beta + g(Z) + \epsilon, \quad E (\epsilon | D, Z) = 0,
$$

using the **double machine learning** approach. 

For $\tilde Y = Y- E(Y|Z)$ and $\tilde D= D- E(D|Z)$, we can write
$$
\tilde Y = \alpha \tilde D + \epsilon, \quad E (\epsilon |\tilde D) =0.
$$

Using cross-fitting, we employ modern regression methods
to build estimators $\hat \ell(Z)$ and $\hat m(Z)$ of $\ell(Z):=E(Y|Z)$ and $m(Z):=E(D|Z)$ to obtain the estimates of the residualized quantities:

$$
\tilde Y_i = Y_i  - \hat \ell (Z_i),   \quad \tilde D_i = D_i - \hat m(Z_i), \quad \text{ for each } i = 1,\dots,n.
$$

Finally, using ordinary least squares of $\tilde Y_i$ on $\tilde D_i$, we obtain the 
estimate of $\beta$.

- First we create the base fuction to use in the __DML__

In [86]:
def DML2_for_PLM(x, d, y, dreg, yreg, nfold):
    
    # Num ob observations
    nobs = x.shape[0]
    
    # Define folds indices 
    list_1 = [*range(0, nfold, 1)]*nobs
    sample = np.random.choice(nobs,nobs, replace=False).tolist()
    foldid = [list_1[index] for index in sample]

    # Create split function(similar to R)
    def split(x, f):
        count = max(f) + 1
        return tuple( list(itertools.compress(x, (el == i for el in f))) for i in range(count) ) 

    # Split observation indices into folds 
    list_2 = [*range(0, nobs, 1)]
    I = split(list_2, foldid)
    
    # Create array to save errors 
    dtil = np.zeros( len(x) ).reshape( len(x) , 1 )
    ytil = np.zeros( len(x) ).reshape( len(x) , 1 )
    
    # loop to save results
    for b in range(0,len(I)):
    
        # Split data - index to keep are in mask as booleans
        include_idx = set(I[b])  #Here should go I[b] Set is more efficient, but doesn't reorder your elements if that is desireable
        mask = np.array([(i in include_idx) for i in range(len(x))])

        # Lasso regression, excluding folds selected 
        dfit = dreg(x[~mask,], d[~mask,])
        yfit = yreg(x[~mask,], y[~mask,])

        # predict estimates using the 
        dhat = dfit.predict( x[mask,] )
        yhat = yfit.predict( x[mask,] )

        # save errors  
        dtil[mask] =  d[mask,] - dhat.reshape( len(I[b]) , 1 )
        ytil[mask] = y[mask,] - yhat.reshape( len(I[b]) , 1 )
        print(b, " ")
    
    # Create dataframe 
    data_2 = pd.DataFrame(np.concatenate( ( ytil, dtil), axis = 1), columns = ['ytil','dtil'])
   
    # OLS clustering at the County level
    model = "ytil ~ dtil"
    baseline_ols = smf.ols(model , data = data_2 ).fit()
    coef_est = baseline_ols.summary2().tables[1]['Coef.']['dtil']
    se = baseline_ols.summary2().tables[1]['Std.Err.']['dtil']
    
    Final_result = { 'coef_est' : coef_est , 'se' : se , 'dtil' : dtil , 'ytil' : ytil }

    print("Coefficient is {}, SE is equal to {}".format(coef_est, se))
    
    return Final_result

<div class="alert alert-block alert-success">
<b>Success:</b> function done
</div>

- Now that we have the function, the next step is to use the differents machine learning method 

<div class="alert alert-block alert-info">
<b>Tip:</b> we have to covert our main variables to matrix</div>

In [87]:
#as matrix
y = y.to_numpy().reshape( len(y) , 1 )
d = d.to_numpy().reshape( len(y) , 1 )
x = x.to_numpy()

## 3. DML using Lasso to predict y an d


### 3.1 Lasso Using scikit-learn


In [88]:
def dreg(x,d):
    alpha=0.00000001
    result = linear_model.Lasso(alpha = alpha).fit(x, d)
    return result

def yreg(x,y):
    alpha=0.00000001
    result = linear_model.Lasso(alpha = alpha).fit(x, y)
    return result

DML2_lasso = DML2_for_PLM(x, d, y, dreg, yreg, 10)

0  
1  
2  
3  
4  
5  
6  
7  
8  
9  
Coefficient is 0.02286307636348808, SE is equal to 0.013254036125334325


This can be interpret as if the initial wealth in crease in 1% then the outcome increase by 0.022%

###  3.2 Lasso using hdmy 

In [116]:
import hdmpy
from statsmodels.tools import add_constant

In [119]:
class rlasso_hdmy:
    
    def __init__(self, post ):
        self.post = post
       
    def fit( self, X, Y ):
        
        self.X = X
        self.Y = Y
        
        # Standarization of X and Y
        self.rlasso_model = hdmpy.rlasso( X , Y , post = self.post )                
        return self
    
    def predict( self , X_1 ):
        self.X_1 = X_1
        beta = self.rlasso_model.est['coefficients'].to_numpy()
        
        if beta.sum() == 0:
            prediction = np.repeat( self.rlasso_model.est['intercept'] , self.X_1.shape[0] )
        
        else:
            prediction = ( add_constant( self.X_1 , has_constant = 'add') @ beta ).flatten()
                
        return prediction

In [120]:
# Post = false
def dreg(x, d):
    result = rlasso_hdmy( post = False ).fit( x , d )
    return result

def yreg(x,y):
    result = rlasso_hdmy( post = False ).fit( x , y )
    return result

DML2_lasso_hdmpy = DML2_for_PLM(x, d, y, dreg, yreg, 10)

0  
1  
2  
3  
4  
5  
6  
7  
8  
9  
Coefficient is -0.03164280324542696, SE is equal to 0.014687979114759876


This can be interpret as if the initial wealth in crease in 1% then the outcome decrease by -0.0316%

## 4. DML using Post-Lasso to predict y an d

### 4.1 Post - Lasso Using scikit-learn


In [89]:
class Lasso_post:
    
    def __init__(self, alpha ):
        self.alpha = alpha

        
    def fit( self, X, Y ):
        self.X = X
        self.Y = Y
        lasso = linear_model.Lasso( alpha = self.alpha ).fit( X , Y )
        model = SelectFromModel( lasso , prefit = True )
        X_new = model.transform( X )
        # Gettin indices from columns which has variance for regression
        index_X = model.get_support()
        
        self.index = index_X
        new_x = X[ : ,  index_X ]
        
        lasso2 = linear_model.Lasso( alpha = self.alpha ).fit( new_x , Y )
        self.model = lasso2
        
        return self
    
    def predict( self , X ):
        
        dropped_X = X[ : , self.index ]
        
        predictions = self.model.predict( dropped_X )
        
        return predictions

In [90]:
def dreg(x,d):
    alpha=0.00000001
    result = Lasso_post( alpha = alpha ).fit( x , d )
    return result

def yreg( x , y ):
    alpha = 0.00000001
    result = Lasso_post( alpha = alpha ).fit( x , y )
    return result

DML2_lasso_post = DML2_for_PLM(x, d, y, dreg, yreg, 10)

0  
1  
2  
3  
4  
5  
6  
7  
8  
9  
Coefficient is 0.02133316532733849, SE is equal to 0.013136859271364234


This can be interpret as if the initial wealth in crease in 1% then the outcome increase by 0.0213%

### 4.2 Post - Lasso using hdmy

In [121]:
import hdmpy
from statsmodels.tools import add_constant

In [122]:
class rlasso_hdmy:
    
    def __init__(self, post ):
        self.post = post
       
    def fit( self, X, Y ):
        
        self.X = X
        self.Y = Y
        
        # Standarization of X and Y
        self.rlasso_model = hdmpy.rlasso( X , Y , post = self.post )                
        return self
    
    def predict( self , X_1 ):
        self.X_1 = X_1
        beta = self.rlasso_model.est['coefficients'].to_numpy()
        
        if beta.sum() == 0:
            prediction = np.repeat( self.rlasso_model.est['intercept'] , self.X_1.shape[0] )
        
        else:
            prediction = ( add_constant( self.X_1 , has_constant = 'add') @ beta ).flatten()
                
        return prediction

In [124]:
# Post = True
def dreg(x, d):
    result = rlasso_hdmy( post = True ).fit( x , d )
    return result

def yreg(x,y):
    result = rlasso_hdmy( post = True ).fit( x , y )
    return result

DML2_lasso_post_hdmpy = DML2_for_PLM(x, d, y, dreg, yreg, 10)

0  
1  
2  
3  
4  
5  
6  
7  
8  
9  
Coefficient is -0.043526751164098994, SE is equal to 0.01356072557603484


This can be interpret as if the initial wealth in crease in 1% then the outcome decrease by -0.0435%

## 5. DML using Elastic Net

In [91]:
class standard_skl_model:
    
    def __init__(self, model ):
        self.model = model
       
    def fit( self, X, Y ):
        
        # Standarization of X and Y
        self.scaler_X = StandardScaler()
        self.scaler_X.fit( X )
        std_X = self.scaler_X.transform( X )
                
        self.model.fit( std_X , Y )
                
        return self
    
    def predict( self , X ):
        
        self.scaler_X = StandardScaler()
        self.scaler_X.fit( X )
        std_X = self.scaler_X.transform( X )
        
        prediction = self.model.predict( std_X )
        
        return prediction

In [92]:
# DML with cross-validated Elastic Net:
def dreg(x,d):
    result = standard_skl_model( ElasticNetCV( cv = 10 , random_state = 0 , l1_ratio = 0.5, max_iter = 100000 ) ).fit( x, d )
    return result

def yreg(x,y):
    result = standard_skl_model( ElasticNetCV( cv = 10 , random_state = 0 , l1_ratio = 0.5, max_iter = 100000 ) ).fit( x, y )
    return result

DML2_elnet = DML2_for_PLM(x, d, y, dreg, yreg, 10 )

0  
1  
2  
3  
4  
5  
6  
7  
8  
9  
Coefficient is -0.022617689027352565, SE is equal to 0.010330550242415635


This can be interpret as if the initial wealth in crease in 1% then the outcome decrease by -0.0226%

## 6. DML using Ridge to predict

In [65]:
#DML with cross-validated Ridge:
def dreg(x,d):
    result = standard_skl_model( ElasticNetCV( cv = 10 ,  random_state = 0 , l1_ratio = 0.0001 ) ).fit( x, d )
    return result

def yreg(x,y):
    result = standard_skl_model( ElasticNetCV( cv = 10 , random_state = 0 , l1_ratio = 0.0001 ) ).fit( x, y )
    return result

DML2_ridge = DML2_for_PLM(x, d, y, dreg, yreg, 10)

0  
1  
2  
3  
4  
5  
6  
7  
8  
9  
Coefficient is -0.01691434758337521, SE is equal to 0.009311024465275054


This can be interpret as if the initial wealth in crease in 1% then the outcome decrease by -0.0169%

## 7. DML using Random Forest

In [67]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [68]:
#DML with Random Forest:
def dreg(x,d):
    result = RandomForestRegressor( random_state = 0 , n_estimators = 500 , max_features = 60 , n_jobs = 4 , min_samples_leaf = 5 ).fit( x, d )
    return result

def yreg(x,y):
    result = RandomForestRegressor( random_state = 0 , n_estimators = 500 , max_features = 60 , n_jobs = 4 , min_samples_leaf = 5 ).fit( x, y )
    return result

DML2_RF = DML2_for_PLM(x, d, y, dreg, yreg, 10)   # set to 2 due to computation time

0  
1  
2  
3  
4  
5  
6  
7  
8  
9  
Coefficient is -0.035561338081183225, SE is equal to 0.01224899142510313


This can be interpret as if the initial wealth in crease in 1% then the outcome decrease by -0.0355%

***

- Now we can join all the results for every method

In [125]:
mods = [DML2_lasso, DML2_lasso_hdmpy, DML2_lasso_post ,DML2_lasso_post_hdmpy, DML2_ridge, DML2_elnet, DML2_RF]
mods_name = ["DML2_lasso", 'DML2_lasso_hdmy',"DML2_lasso_post",'DML2_lasso_post_hdmpy', 'DML2_ridge', 'DML2_elnet', 'DML2_RF']

def mdl( model , model_name ):
    
    RMSEY = np.sqrt( np.mean( model[ 'ytil' ] ) ** 2 ) # I have some doubts about these equations...we have to recheck
    RMSED = np.sqrt( np.mean( model[ 'dtil' ] ) ** 2 ) # I have some doubts about these equations...we have to recheck
    
    result = pd.DataFrame( { model_name : [ RMSEY , RMSED ]} , index = [ 'RMSEY' , 'RMSED' ])
    return result

RES = [ mdl( model , name ) for model, name in zip( mods , mods_name ) ]
    

pr_Res = pd.concat( RES, axis = 1)

pr_Res

,DML2_lasso,DML2_lasso_hdmy,DML2_lasso_post,DML2_lasso_post_hdmpy,DML2_ridge,DML2_elnet,DML2_RF
RMSEY,0.002094,0.001052,0.002321,0.000415,7.709882e-18,2.937465e-17,0.000511
RMSED,0.003916,0.013437,0.000987,0.002541,3.552714e-16,1.480297e-15,0.012705


It looks like the best method fro predicting $Y$ is Ridge and for $D$ is Ridge too. 

## 8. Run the best method i.e. the best combination of methods to predict y an d

In [130]:
def dreg(x,d):
    result = standard_skl_model(  ElasticNetCV(cv = 10 , random_state = 0 , alphas = [0]) ).fit( x, d )
    return result


def yreg(x,y):
    result = standard_skl_model( ElasticNetCV( cv = 10 ,  random_state = 0 , l1_ratio = 0.0001 ) ).fit( x, y )
    return result

DML2_best = DML2_for_PLM(x, d, y , dreg, yreg, 10)



0  
1  
2  
3  
4  
5  
6  
7  
8  
9  
Coefficient is -0.005285590458468861, SE is equal to 0.007555802610939796


We can coclude that country outcome is negative related to the initial wealth. This can be interpret as if the initial wealth in crease in 1% then the outcome decrease by -0.0052%

##  9. Show your results in a table as we did in the lab.

In [127]:
table = np.zeros( ( 10 , 2 ))
table[ 0 , 0] = baseline_ols.summary2().tables[1]['Coef.']['d']
table[ 1 , 0] = control_ols.summary2().tables[1]['Coef.']['d']
table[ 2 , 0] = DML2_lasso['coef_est']
table[ 3 , 0] = DML2_lasso_hdmpy['coef_est']
table[ 4 , 0] = DML2_lasso_post['coef_est']
table[ 5 , 0] = DML2_lasso_post_hdmpy['coef_est']
table[ 6 , 0] = DML2_ridge['coef_est']
table[ 7 , 0] = DML2_elnet['coef_est']
table[ 8 , 0] = DML2_RF['coef_est']
table[ 9 , 0] = DML2_best['coef_est']
table[ 0 , 1] = baseline_ols.summary2().tables[1]['Std.Err.']['d']
table[ 1 , 1] = control_ols.summary2().tables[1]['Std.Err.']['d']
table[ 2 , 1] = DML2_lasso['se']
table[ 3 , 1] = DML2_lasso_hdmpy['se']
table[ 4 , 1] = DML2_lasso_post['se']
table[ 5 , 1] = DML2_lasso_post_hdmpy['se']
table[ 6 , 1] = DML2_ridge['se']
table[ 7 , 1] = DML2_elnet['se']
table[ 8 , 1] = DML2_RF['se']
table[ 9 , 1] = DML2_best['se']

In [128]:
table = pd.DataFrame( table , index = [ "Baseline OLS", "OLS with controls", "Lasso", \
'Lasso using hdmy', "Post-Lasso",'Post-Lasso using hdmy',"CV Elnet", "CV Ridge", "Random Forest", "Best" ] , \
            columns = ["Estimate","Standard Error"] )
table.round( 3 )

,Estimate,Standard Error
Baseline OLS,0.001,0.006
OLS with controls,-0.009,0.030
Lasso,0.023,0.013
Lasso using hdmy,-0.032,0.015
Post-Lasso,0.021,0.013
Post-Lasso using hdmy,-0.044,0.014
CV Elnet,-0.017,0.009
CV Ridge,-0.023,0.010
Random Forest,-0.036,0.012
Best,0.011,0.008
